In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
import sys

In [ ]:
sys.executable

# Load data

In [ ]:
fft_size = 2**14
sample_size = 200_000
sec_per_fft = fft_size / sample_size
freq_per_bin = sample_size / fft_size

In [ ]:
power_spectrum_file = "/home/thomas/projects/nilm_at_fair/training_data/training_data_2022-10-12/MixedOnOff-FFTVolatage-200KS-FFTSize16k_DAT_2022-10-12"
power_spectrum = np.fromfile(power_spectrum_file, dtype=np.float32).reshape((-1,fft_size))
power_spectrum_real = power_spectrum[:,0:int(fft_size/2)]
frequencies = (np.arange(fft_size / 2) * freq_per_bin + freq_per_bin / 2) * 0.001

In [ ]:
freq_per_bin

In [ ]:
power_spectrum_real.shape

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(111)

ax.plot(frequencies,
        power_spectrum_real[0])
ax.grid(True)
ax.set_xlabel('Frequency[kHz]')

In [ ]:
fig = plt.figure(dpi=500, figsize=(16, 9))
ax = fig.add_subplot(111)

im = ax.imshow(X=power_spectrum_real.T,
               aspect='auto',
               interpolation='none',
               origin='lower')

# add Frequency labels
y_labels = np.arange(0, 100_001, 10_000)
y_label_positions = y_labels / freq_per_bin
ax.set_yticks(ticks=y_label_positions, labels=y_labels / 1_000)
ax.set_ylabel("Frequency [kHz]")

# add time labels
t_max = power_spectrum_real.shape[0] * sec_per_fft
x_labels = np.arange(0, t_max, 10)
x_tick_positions = x_labels / sec_per_fft
ax.set_xticks(x_tick_positions, x_labels)
ax.set_xlabel('Time [s]')

fig.colorbar(im, ax=ax)

ax.set_title('Current Spectrum Versus Time')

fig.savefig('/home/thomas/projects/nilm_at_fair/mixed_voltage_spectrum_vs_time.png')


In [ ]:
t_max = power_spectrum_real.shape[0] * sec_per_fft
x_labels = np.arange(0, t_max, 10)
x_tick_positions = x_labels / sec_per_fft
ax.set_xticks(x_tick_positions, x_labels)

In [ ]:
ax

In [ ]:
f = np.fromfile(open("/home/thomas/projects/nilm_at_fair/gnu_radio/gnu_radio/fft_file_sink_test"), dtype=np.float32)

In [ ]:
f.shape

In [ ]:
f_reshaped = f.reshape((-1,1000))

In [ ]:
f_reshaped.shape

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(111)
ax.plot(f_reshaped[1])
ax.grid()

In [ ]:
10150 / 1000

In [ ]:
import glob

In [ ]:
active_power_file = "/home/thomas/projects/nilm_at_fair/training_data/training_data_2022-10-12/halo/HaloOnOff-P-200KS_DAT_2022-10-12"
active_power = np.fromfile(active_power_file, dtype=np.float32)

In [ ]:
active_power.shape[0] / 192

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(active_power)

In [ ]:
active_power.shape

In [ ]:
type(ax)

In [ ]:
type(fig)

# Test modules

In [6]:
import sys
import matplotlib.pyplot as plt
import numpy as np

In [7]:
sys.path.append("/home/thomas/projects/nilm_at_fair/repository/")

In [8]:
from src.pulsed_power_ml.model_framework.data_io import read_training_files
from src.pulsed_power_ml.model_framework.visualizations import add_contour_plot, plot_data_point_array

In [11]:
fft_size = 2**14
data_point_array = read_training_files(
    "/home/thomas/projects/nilm_at_fair/training_data/training_data_2022-10-12/tube/",
    fft_size=fft_size,
    sample_size=200_000,
)

In [ ]:
fig = plot_data_point_array(data_point_array,
                            fft_size)

fig.savefig("/home/thomas/projects/nilm_at_fair/training_data/training_data_2022-10-12/tube/figures/test.png")